<a href="https://colab.research.google.com/github/dmitryt4/Practical-Data-Analysis-Second-Edition/blob/master/TestPlayWright.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!sudo apt-get update && sudo apt-get install -y libwoff1 libopus0 libwebp6 libwebpdemux2 libenchant1c2a libgudev-1.0-0 libsecret-1-0 libhyphen0 libgdk-pixbuf2.0-0 libegl1 libnotify4 libxslt1.1 libevent-2.1-7 libgles2 libvpx6 libxcomposite1 libatk1.0-0 libatk-bridge2.0-0 libepoxy0 libgtk-3-0 libharfbuzz-icu0

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,454 kB]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,224 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packa

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install --upgrade --quiet playwright
!pip install --upgrade --quiet crawl4ai
!pip install --upgrade --quiet lxml
!pip install nest_asyncio
!pip install aiofiles
!playwright install

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.5/409.5 kB 24.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.36
    Uninstalling SQLAlchemy-2.0.36:
      Successfully uninstalled SQLAlchemy-2.0.36
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.19
    Uninstalling langchain-core-0.3.19:
      Successfully uninstalled langchain-core-0.3.19
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.7
    Uninstalling langchain-0.3.7:
      Successfully uninstalled langchain-0.3.7
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 MB 18.7 MB/s eta 0:00:00
     ━━━━

In [ ]:
import asyncio
import nest_asyncio
from crawl4ai import AsyncWebCrawler
from crawl4ai.extraction_strlategy import JsonCssExtractionStrategy, LLMExtractionStrategy
import json
import time
from pydantic import BaseModel, Field
import os

nest_asyncio.apply()

In [ ]:
import os
from google.colab import userdata
"""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ['LANGCHAIN_API_KEY'] = userdata.get('langchain_api_key')
"""
os.environ["OPENAI_API_KEY"] = userdata.get('openai_api_key')

In [ ]:
from google.colab import drive
drive.mount('/storage')
persist_dir = "/storage/MyDrive/storage"

Mounted at /storage


In [ ]:
async def extract_events():
    schema = {
        "name": "Chattanooga Events",
        "baseSelector": "div.events---all div.flex-4-columns.events.w-dyn-item",
        "fields":[
            {
                "name": "event",
                "selector": "h3.event-title",
                "type": "text"
            },
            {
                "name": "url",
                "selector": "a[href]",
                "type": "attribute",
                "attribute": "href"
            }
        ]
    }
    extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url="https://www.cha.guide/events",
            extraction_strategy=extraction_strategy,
            bypass_cache=False
        )
        assert result.success, "Failed to crawl the page"
        ch_events = json.loads(result.extracted_content)
        print(len(ch_events))
        os.makedirs(os.path.dirname(f"{persist_dir}/crawl4ai/"), exist_ok=True)
        with open(f"{persist_dir}/crawl4ai/ch_events.json", "w") as f:
            json.dump(ch_events, f, indent=2)

In [ ]:
await extract_events()

[INIT].... → Crawl4AI 0.3.742
[FETCH]... ↓ https://www.cha.guide/events... | Status: True | Time: 2.49s
[SCRAPE].. ◆ Processed https://www.cha.guide/events... | Time: 656ms
[EXTRACT]. ■ Completed for https://www.cha.guide/events... | Time: 0.37392619000002014s
[COMPLETE] ● https://www.cha.guide/events... | Status: True | Total: 4.88s
80


In [ ]:
def get_event_urls(file_path):
    try:
        with open(file_path, "r") as f:
            events = json.load(f)

        # Extract URLs
        urls = [event.get("url") for event in events if "url" in event]

        return urls
    except FileNotFoundError:
        print(f"The file {file_path} does not exist.")
        return []
    except json.JSONDecodeError:
        print(f"Error decoding JSON from the file {file_path}.")
        return []

In [ ]:

event_schema = {
  "name": "Chattanooga Event",
  "baseSelector": "body",
  "fields": [
          {
            "name": "event",
            "selector": "div.blue h1.white",
            "type": "text"
          },
          {
            "name": "date",
            "selector": "div.standard-section div.flex-table.w-row div.info-block>div:nth-child(1) p",
            "type": "text"
          },
          {
            "name": "time",
            "selector": "div.standard-section div.flex-table.w-row div.info-block>div:nth-child(2) p",
            "type": "text"
          },
          {
              "name": "location",
              "selector": "div.standard-section div.flex-table.w-row div.info-block>div:nth-child(3) p",
              "type": "text"
          },
          {
              "name": "organaizer",
              "selector": "div.standard-section div.flex-table.w-row div.info-block>div:nth-child(5) p",
              "type": "text"
          },
          {
              "name": "price",
              "selector": "div.standard-section div.flex-table.w-row div.info-block>div:nth-child(6)",
              "type": "text"
          },
          {
              "name": "description",
              "selector": "div.standard-section div.narrow div.flex-table.w-row div.flex-2-columns",
              "type": "text"
          },
          {
              "name": "url",
              "selector": "div.standard-section div.centered-in-mobile a[href]",
              "type": "attribute",
              "attribute": "href"
          },
      ]
}

In [ ]:
extraction_strategy = JsonCssExtractionStrategy(event_schema, verbose=True)
async def crawl_event(url):
    f_path = f"{persist_dir}/crawl4ai/events.json"
    extraction_strategy = JsonCssExtractionStrategy(event_schema, verbose=True)
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(
            url=f"https://www.cha.guide{url}",
            extraction_strategy=extraction_strategy,
            bypass_cache=True
        )
        if os.path.exists(f_path):
            with open(f_path, "r") as f:
                try:
                    events = json.load(f)
                except json.JSONDecodeError:
                    events = []  # Start with empty list if JSON is malformed or empty
        else:
            events = []
        print(result.extracted_content)
        # Append the new data
        new_event = json.loads(result.extracted_content)
        events.extend(new_event)
        print(f"events object: {len(events)}")
        # Write all data back to the file
        with open(f_path, "w") as f:
            json.dump(events, f, indent=2)

In [ ]:
event_urls = get_event_urls(f"{persist_dir}/crawl4ai/ch_events.json")
for event_url in event_urls:
  await crawl_event(event_url)

[INIT].... → Crawl4AI 0.3.742


<ipython-input-8-1645a1e52e9f>:6: DeprecationWarning: Cache control boolean flags are deprecated and will be removed in version X.X.X. Use 'cache_mode' parameter instead. Examples:
- For bypass_cache=True, use cache_mode=CacheMode.BYPASS
- For disable_cache=True, use cache_mode=CacheMode.DISABLED
- For no_cache_read=True, use cache_mode=CacheMode.WRITE_ONLY
- For no_cache_write=True, use cache_mode=CacheMode.READ_ONLY
Pass warning=False to suppress this warning.
  result = await crawler.arun(


[FETCH]... ↓ https://www.cha.guide/events/retro-halloween-party... | Status: True | Time: 2.40s
[SCRAPE].. ◆ Processed https://www.cha.guide/events/retro-halloween-party... | Time: 40ms
[EXTRACT]. ■ Completed for https://www.cha.guide/events/retro-halloween-party... | Time: 0.04218697600003907s
[COMPLETE] ● https://www.cha.guide/events/retro-halloween-party... | Status: True | Total: 2.52s
[
    {
        "event": "Retro Halloween Party",
        "date": "Nov 26",
        "time": "7pm-2:30am",
        "location": "Reagan's Retro Bar",
        "organaizer": "Reagan's Retro Bar",
        "price": "Cost$23.18",
        "description": "Get ready to travel back in time and celebrate Halloween in retro style with our groovy costumes, funky music, and nostalgic vibes!Reagan's Retro Bar are throwing the ultimate retro Halloween party from 7 PM to 2:45 AM that’s sure to be filled with chills, thrills, and killer costumes. 🎃Costume Contest: Strut your stuff and compete for over $500 in cash and

In [ ]:
import aiofiles
async def simple_crawl():
    async with AsyncWebCrawler(verbose=True) as crawler:
        result = await crawler.arun(url="https://www.cha.guide/events")
        print(result.markdown[:1000])
        async with aiofiles.open('crawl_result.md', 'w') as f:
            await f.write(result.markdown)
await simple_crawl()

[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://www.cha.guide/events using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://www.cha.guide/events successfully!
[LOG] 🚀 Crawling done for https://www.cha.guide/events, success: True, time taken: 2.20 seconds
[LOG] 🚀 Content extracted for https://www.cha.guide/events, success: True, time taken: 1.69 seconds
[LOG] 🔥 Extracting semantic blocks for https://www.cha.guide/events, Strategy: AsyncWebCrawler
[LOG] 🚀 Extraction done for https://www.cha.guide/events, time taken: 1.69 seconds.


Exception: Browser.close: Connection closed while reading from the driver

In [ ]:
async def extract_structured_data_using_css_extractor():
  print("\n--- Using JsonCssExtractionStrategy for Fast Structured Output ---")

  # Define the extraction schema
  schema = {
      "name": "Chattanooga Events",
      "baseSelector": "div.flex-4-columns.events.w-dyn-item", #search-bar > div > div.listing-div > div.events---all > div.flex-wrapper.w-dyn-list > div.flex-table.w-dyn-items > div:nth-child(1)
      "fields": [
          {
              "name": "event",
              "selector": "h3.event-title",
              "type": "text"
          },
          {
              "name": "url",
              "selector": "a[href]",
              "type": "attribute",
              "attribute": "href"
          },
          {
              "name": "date",
              "selector": "div.smaller-text.bottom-margin---10px",
              "type": "text"
          },
          {
              "name": "category",
              "selector": "div.event-title-and-date-div.bottom-margin---10px > div:nth-child(1) > div:nth-child(3) > div:nth-child(4) > div.in-line.smaller-text",
              "type": "text"
          },
          {
              "name": "description",
              "selector": "div.truncate.bottom-margin---5px",
              "type": "text"
          },
          {
              "name": "location",
              "selector": "div.location-2",
              "type": "text"
          },
      ]
  }
  extraction_strategy = JsonCssExtractionStrategy(schema, verbose=True)
  async with AsyncWebCrawler(verbose=True) as crawler:
    result = await crawler.arun(
        url="https://www.cha.guide/events",
        extraction_strategy=extraction_strategy,
        bypass_cache=True
    )
    assert result.success, "Failed to crawl the page"
    ch_events = json.loads(result.extracted_content)
    with open("ch_events.json", "w") as f:
      json.dump(ch_events, f, indent=2)
await extract_structured_data_using_css_extractor()


--- Using JsonCssExtractionStrategy for Fast Structured Output ---
[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://www.cha.guide/events using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://www.cha.guide/events successfully!
[LOG] 🚀 Crawling done for https://www.cha.guide/events, success: True, time taken: 3.67 seconds
[LOG] 🚀 Content extracted for https://www.cha.guide/events, success: True, time taken: 3.27 seconds
[LOG] 🔥 Extracting semantic blocks for https://www.cha.guide/events, Strategy: AsyncWebCrawler
[LOG] 🚀 Extraction done for https://www.cha.guide/events, time taken: 4.42 seconds.


In [ ]:
schema = {
      "name": "Chattanooga Events",
      "fields": [
          {
              "name": "event",
             # "selector": "h3.event-title",
              "type": "text"
          },
          {
              "name": "url",
              "selector": "a[href]",
              "type": "attribute",
              "attribute": "href"
          },
          {
              "name": "date",
              "selector": "div.smaller-text.bottom-margin---10px",
              "type": "text"
          },
          {
              "name": "category",
              #"selector": "div.event-title-and-date-div.bottom-margin---10px > div:nth-child(1) > div:nth-child(3) > div:nth-child(4) > div.in-line.smaller-text",
              "type": "text"
          },
          {
              "name": "description",
              "selector": "div.truncate.bottom-margin---5px",
              "type": "text"
          },
          {
              "name": "location",
              "selector": "div.location-2",
              "type": "text"
          },
      ]
  }

In [ ]:
from crawl4ai.extraction_strategy import LLMExtractionStrategy
async def get_events():
  url = "https://www.cha.guide/events"
  instruction="""
                 From the crawled content extract first 2 events that will take place in Chattanooga.
                 For each event you should extract following details:the name of the event,
                 the full url to the page of the event, the event date, the event vidible category, the event description
                 and the event location.
                 You should follow some rules for extracting the content:
                 For the category field you should extract text from the element that you can find with CSS selector:
                 div:not(.w-condition-invisible) > .event---category-circle + .in-line.smaller-text
                 The url field should contain the full url as following: 'https://cha.guide/events/...'
              """
  instruction1 = """
                    From the crawled content extract the text, that you find with CSS selector: body > div.nav-menu > div.menu-div > div:nth-child(3) > a:nth-child(2) > div > div
                 """
  extraction_strategy = LLMExtractionStrategy(
                                              provider="openai/gpt-4o",
                                              api_key=os.environ["OPENAI_API_KEY"],
                                              schema=schema,
                                              extraction_type="schema",
                                              instruction=instruction,
                                              )
  async with AsyncWebCrawler(verbose=True) as crawler:
    result = await crawler.arun(
                                url=url,
                                word_count_threshold=1,
                                extraction_strategy=extraction_strategy,
                                bypass_cache=False
                               )
    events = json.loads(result.extracted_content)
    print(len(events))
    with open("events.json", "w") as f:
      json.dump(events, f, indent=2)
await get_events()

[LOG] 🌤️  Warming up the AsyncWebCrawler
[LOG] 🌞 AsyncWebCrawler is ready to crawl
[LOG] 🕸️ Crawling https://www.cha.guide/events using AsyncPlaywrightCrawlerStrategy...
[LOG] ✅ Crawled https://www.cha.guide/events successfully!
[LOG] 🚀 Crawling done for https://www.cha.guide/events, success: True, time taken: 4.04 seconds
[LOG] 🚀 Content extracted for https://www.cha.guide/events, success: True, time taken: 2.69 seconds
[LOG] 🔥 Extracting semantic blocks for https://www.cha.guide/events, Strategy: AsyncWebCrawler
[LOG] Call LLM for https://www.cha.guide/events - block index: 0
[LOG] Call LLM for https://www.cha.guide/events - block index: 1
[LOG] Call LLM for https://www.cha.guide/events - block index: 2
[LOG] Call LLM for https://www.cha.guide/events - block index: 3
[LOG] Extracted 2 blocks from URL: https://www.cha.guide/events block index: 3


/usr/local/lib/python3.10/dist-packages/pydantic/main.py:390: UserWarning: Pydantic serializer warnings:
  Expected `PromptTokensDetails` but got `dict` with value `{'audio_tokens': None, 'cached_tokens': 0}` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


[LOG] Extracted 2 blocks from URL: https://www.cha.guide/events block index: 2
[LOG] Extracted 2 blocks from URL: https://www.cha.guide/events block index: 0
[LOG] Extracted 2 blocks from URL: https://www.cha.guide/events block index: 1
[LOG] 🚀 Extraction done for https://www.cha.guide/events, time taken: 13.42 seconds.
8


In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import (
    create_async_playwright_browser,
    create_sync_playwright_browser,
)

In [ ]:
import asyncio
async def setup_toolkit():
      # Create only the async browser
  async_browser = create_async_playwright_browser()

    # Initialize the toolkit with only the async browser
  toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
  tools = toolkit.get_tools()
  return tools

loop = asyncio.get_event_loop()


tools = loop.run_until_complete(setup_toolkit())

print(tools)

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>), NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>), NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>), ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>), ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>), GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chr

In [ ]:
#sync_browser = create_sync_playwright_browser()
async_browser = create_async_playwright_browser()
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()
tools

[ClickTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>),
 NavigateTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>),
 NavigateBackTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>),
 ExtractTextTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>),
 ExtractHyperlinksTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-1134/chrome-linux/chrome> version=129.0.6668.29>),
 GetElementsTool(async_browser=<Browser type=<BrowserType name=chromium executable_path=/root/.cache/ms-playwright/chromium-113